# __3. 3. Conversion Prediction__

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

In [2]:
# Load pickles
df_train = pd.read_pickle("df_train.pkl")
df_test = pd.read_pickle("df_test.pkl")
df_valid = pd.read_pickle("df_valid.pkl")

In [3]:
# Train data X/y 
X_train = df_train.drop('y',axis=1).values
y_train = df_train['y'].values

# Test data X/y 
X_test = df_test.drop('y',axis=1).values
y_test = df_test['y'].values

# Validation data X/y 
X_valid = df_valid.drop('y',axis=1).values
y_valid = df_valid['y'].values

***

## 1. Logistic Regression

***

#### 1.1 Base Model

In [53]:
# Create Pipeline
logreg_base = Pipeline ([
        (("enc", OneHotEncoder(handle_unknown = 'ignore'))),
        ("scaler", StandardScaler(with_mean=False)),
        ("logreg", LogisticRegression(multi_class='ovr', solver='liblinear'))
])

# Fit Pipeline
logreg_base.fit(X_train, y_train)

# Accuracy on test set
accuracy_logreg = logreg_base.score(X_valid, y_valid)
print('accuracy logistic regression: {:.2f}%'.format(accuracy_logreg))

accuracy logistic regression: 0.90%


#### 1.2 Tuned model 

In [54]:
# Create Pipeline
logreg_cv = Pipeline ([
        (("enc", OneHotEncoder(handle_unknown = 'ignore'))),
        ("scaler", StandardScaler(with_mean=False)),
        ("logreg", LogisticRegression(multi_class='ovr', solver='liblinear'))
])

# Create cross-validation
grid = {'logreg__C': np.logspace(-5, 5, num=10)} # Tune C parameter
grid_cv = GridSearchCV(logreg_cv, grid, cv=5, return_train_score=True, n_jobs=-1)

# Fit model
grid_cv.fit(X_valid, y_valid)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('enc', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)), ('scaler', StandardScaler(copy=True, with_mean=False, with_std=True)), ('logreg', LogisticRegression(C=1.0, class_weight=None, ...ty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'logreg__C': array([1.00000e-05, 1.29155e-04, 1.66810e-03, 2.15443e-02, 2.78256e-01,
       3.59381e+00, 4.64159e+01, 5.99484e+02, 7.74264e+03, 1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [55]:
cv_results = pd.DataFrame(grid_cv.cv_results_)

#print the most relevant columns
cols = ['mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score', 'param_logreg__C']
tuned_logreg=cv_results[cols].sort_values('mean_test_score', ascending=False)
tuned_logreg

,mean_test_score,std_test_score,mean_train_score,std_train_score,param_logreg__C
2,0.895120,0.003907,0.916181,0.001088,0.0016681
1,0.891721,0.002089,0.898185,0.000463,0.000129155
3,0.889779,0.004543,0.917911,0.000785,0.0215443
4,0.889172,0.005525,0.918002,0.000744,0.278256
5,0.888687,0.005057,0.917911,0.000590,3.59381
6,0.888687,0.005213,0.917911,0.000656,46.4159
7,0.888687,0.005213,0.917911,0.000656,599.484
8,0.888565,0.005323,0.917911,0.000656,7742.64
9,0.888565,0.005323,0.917911,0.000656,100000
0,0.886137,0.000449,0.886775,0.000116,1e-05


Changing the C parameter doesn't seem to improve the performance

In [7]:
# Saving the best peforimng C-parameter
Tuned_C = tuned_logreg['param_logreg__C'].iloc[0]

tuned_logreg = Pipeline ([
        (("enc", OneHotEncoder(handle_unknown = 'ignore'))),
        ("scaler", StandardScaler(with_mean=False)),
        ("logreg", LogisticRegression(multi_class='ovr', solver='liblinear', C=Tuned_C))
])

# Fit the model
tuned_logreg.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('enc', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)), ('scaler', StandardScaler(copy=True, with_mean=False, with_std=True)), ('logreg', LogisticRegression(C=0.0016681005372000592, c...ty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [35]:
tuned_accuracy_logreg = tuned_logreg.score(X_test, y_test)
print('Accuracy tuned logistic regression: {:.2f}%'.format(tuned_accuracy_logreg))

Accuracy tuned logistic regression: 0.90%


In [58]:
from sklearn.metrics import confusion_matrix

# Calculate predictions
y_valid_pred = tuned_logreg.predict(X_valid)

logreg_confusion_matrix = confusion_matrix(y_valid, y_valid_pred)

logreg_confusion_matrix # cannot import confusionmatrixdisplay from scikit - not sure why. 

AttributeError: 'DataFrame' object has no attribute 'predict'

A relative high number of false postivie. 

In [44]:
from sklearn.externals import joblib

# Save estimator
joblib.dump(tuned_logreg, "tuned_logreg.joblib")

['tuned_logreg.joblib']

## 2. Random Forest

#### 2.1 Base Model

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Create Pipeline
ranfor = Pipeline ([
        (("enc", OneHotEncoder(handle_unknown = 'ignore'))),
        ("scaler", StandardScaler(with_mean=False)),
        ("ranfor", RandomForestClassifier(criterion='gini',random_state=0))
])

# Fit Pipeline
ranfor.fit(X_train, y_train)

# Accuracy on test set
accuracy_ranfor = ranfor.score(X_valid, y_valid)
print('accuracy random forest: {:.2f}%'.format(accuracy_ranfor))

/Users/samerre/opt/anaconda3/envs/exts-ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


accuracy random forest: 0.89%


#### 2.2 Tuned Model

In [14]:
# Create Pipeline
ranfor_tuned = Pipeline ([
        (("enc", OneHotEncoder(handle_unknown = 'ignore'))),
        ("scaler", StandardScaler(with_mean=False)),
        ("ranfor", RandomForestClassifier(criterion='gini',random_state=0))
])

grid = {
    "ranfor__n_estimators": [1,5,10,20,50,100,200],
    "ranfor__max_depth": [1,3,5,10,50,None]
}

dtcv = GridSearchCV(ranfor_tuned,grid,cv=5,return_train_score=True)

dtcv.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('enc', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)), ('scaler', StandardScaler(copy=True, with_mean=False, with_std=True)), ('ranfor', RandomForestClassifier(bootstrap=True, class_...ors='warn', n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'ranfor__n_estimators': [1, 5, 10, 20, 50, 100, 200], 'ranfor__max_depth': [1, 3, 5, 10, 50, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [51]:
dtcv.best_params_

{'ranfor__max_depth': 10, 'ranfor__n_estimators': 50}

In [16]:
dtcv.best_score_

0.9010197474910975

In [57]:
# Saving the best peforimng C-parameter
best_max_depth = dtcv.best_params_['ranfor__max_depth']
best_n_estimators = dtcv.best_params_['ranfor__n_estimators']

ranfor_best = Pipeline ([
        (("enc", OneHotEncoder(handle_unknown = 'ignore'))),
        ("scaler", StandardScaler(with_mean=False)),
        ("ranfor", RandomForestClassifier(criterion='gini',random_state=0, n_estimators = best_n_estimators, max_depth = best_max_depth))
])

# Fit the model
ranfor_best.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('enc', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)), ('scaler', StandardScaler(copy=True, with_mean=False, with_std=True)), ('ranfor', RandomForestClassifier(bootstrap=True, class_...imators=50, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False))])

In [ ]:
dtcv_test_predictions = dtcv.predict(X_test)
dtcv_test_predictions

## 3. XGboost

#### 3.1 Base Model

In [10]:
import xgboost

In [11]:
model_xgboost = xgboost.XGBClassifier(learning_rate = 0.1,
                                    max_depth =20,
                                    n_estimators=5000,
                                    subsample=0.5,
                                    colsample_bytree=0.5,
                                    eval_metric="auc",
                                    verbosity=1)

In [13]:
# encode dasets
df_train_enc = pd.get_dummies(df_train)
df_valid_enc = pd.get_dummies(df_valid)

# Train data X/y 
X_train_enc = df_train_enc.drop('y',axis=1).values
y_train_enc = df_train_enc['y'].values

# Test data X/y 
X_valid_enc = df_valid_enc.drop('y',axis=1).values
y_valid_enc = df_valid_enc['y'].values

In [14]:
eval_set = [(X_valid_enc, y_valid_enc)]

model_xgboost.fit(X_train_enc,
                 y_train_enc,
                 early_stopping_rounds=20,
                 eval_set=eval_set,
                 verbose=True)

[0]	validation_0-auc:0.77343
[1]	validation_0-auc:0.77751
[2]	validation_0-auc:0.78344
[3]	validation_0-auc:0.78631
[4]	validation_0-auc:0.78873
[5]	validation_0-auc:0.79139
[6]	validation_0-auc:0.79136
[7]	validation_0-auc:0.79268
[8]	validation_0-auc:0.79189
[9]	validation_0-auc:0.78957
[10]	validation_0-auc:0.78980
[11]	validation_0-auc:0.79044
[12]	validation_0-auc:0.79049
[13]	validation_0-auc:0.79198
[14]	validation_0-auc:0.79149
[15]	validation_0-auc:0.79003
[16]	validation_0-auc:0.79107
[17]	validation_0-auc:0.78975
[18]	validation_0-auc:0.78970
[19]	validation_0-auc:0.78962
[20]	validation_0-auc:0.79275
[21]	validation_0-auc:0.79329
[22]	validation_0-auc:0.79284
[23]	validation_0-auc:0.79246
[24]	validation_0-auc:0.79263
[25]	validation_0-auc:0.79269
[26]	validation_0-auc:0.79260
[27]	validation_0-auc:0.79145
[28]	validation_0-auc:0.79123
[29]	validation_0-auc:0.79184
[30]	validation_0-auc:0.79034
[31]	validation_0-auc:0.79085
[32]	validation_0-auc:0.79067
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.5, eval_metric='auc',
       gamma=0, gpu_id=-1, importance_type='gain',
       interaction_constraints='', learning_rate=0.1, max_delta_step=0,
       max_depth=20, min_child_weight=1, missing=nan,
       monotone_constraints='()', n_estimators=5000, n_jobs=8,
       num_parallel_tree=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.5,
       tree_method='exact', use_label_encoder=True, validate_parameters=1,
       verbosity=1)

In [19]:
from sklearn.metrics import roc_auc_score

y_train_pred = model_xgboost.predict_proba(X_train_enc)[:,1]
y_valid_pred = model_xgboost.predict_proba(X_valid_enc)[:,1]

print('accuracy train set: {:.2f}%'.format(roc_auc_score(y_train_enc, y_train_pred)))
print('accuracy validation set: {:.2f}%'.format(roc_auc_score(y_valid_enc, y_valid_pred)))

accuracy train set: 0.88%
accuracy validation set: 0.79%


#### 3.2 Tuned Model

In [21]:
grid = {
    "learning_rate": [0.02, 0.05, 0.1],
    "max_depth": [2, 3, 4, 5],
    "n_estimators": [1000, 2000, 3000]
}

model_xgboost_tuned = GridSearchCV(model_xgboost, grid, cv=3, return_train_score=True, verbose=4)

model_xgboost_tuned.fit(X_train_enc, y_train_enc)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] learning_rate=0.02, max_depth=2, n_estimators=1000 ..............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/samerre/opt/anaconda3/envs/exts-ml/lib/python3.6/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV]  learning_rate=0.02, max_depth=2, n_estimators=1000, score=0.9039815489196407, total=   4.9s
[CV] learning_rate=0.02, max_depth=2, n_estimators=1000 ..............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.0s remaining:    0.0s


[CV]  learning_rate=0.02, max_depth=2, n_estimators=1000, score=0.9022702440208814, total=   6.6s
[CV] learning_rate=0.02, max_depth=2, n_estimators=1000 ..............


...

[CV]  learning_rate=0.02, max_depth=2, n_estimators=1000, score=0.899963578972927, total=   6.1s
[CV] learning_rate=0.02, max_depth=2, n_estimators=2000 ..............


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   17.8s remaining:    0.0s


[CV]  learning_rate=0.02, max_depth=2, n_estimators=2000, score=0.9009468317552805, total=   9.6s
[CV] learning_rate=0.02, max_depth=2, n_estimators=2000 ..............
[CV]  learning_rate=0.02, max_depth=2, n_estimators=2000, score=0.9021488405973048, total=   9.0s
[CV] learning_rate=0.02, max_depth=2, n_estimators=2000 ..............
[CV]  learning_rate=0.02, max_depth=2, n_estimators=2000, score=0.9002063858200802, total=   9.8s
[CV] learning_rate=0.02, max_depth=2, n_estimators=3000 ..............
[CV]  learning_rate=0.02, max_depth=2, n_estimators=3000, score=0.9003398883224083, total=  13.2s
[CV] learning_rate=0.02, max_depth=2, n_estimators=3000 ..............
[CV]  learning_rate=0.02, max_depth=2, n_estimators=3000, score=0.9022702440208814, total=  17.8s
[CV] learning_rate=0.02, max_depth=2, n_estimators=3000 ..............
[CV]  learning_rate=0.02, max_depth=2, n_estimators=3000, score=0.9006919995143863, total=  20.1s
[CV] learning_rate=0.02, max_depth=3, n_estimators=1000 .

[Parallel(n_jobs=1)]: Done 108 out of 108 | elapsed: 34.5min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.5, eval_metric='auc',
       gamma=0, gpu_id=-1, importance_type='gain',
       interaction_constraints='', learning_rate=0.1, max_delta_step=0,
       max_depth=20, min_child...=0.5,
       tree_method='exact', use_label_encoder=True, validate_parameters=1,
       verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'learning_rate': [0.02, 0.05, 0.1], 'max_depth': [2, 3, 4, 5], 'n_estimators': [1000, 2000, 3000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=4)

In [ ]:
***

## 2.1 K-Means

***